<a href="https://colab.research.google.com/github/tanodino/Summer_School_JOENSUU/blob/main/Joensuu_RS%20/Joensuu_Day02_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pystac_client
!pip install planetary_computer
!pip install stackstac

# STAC access
import pystac_client
import planetary_computer

# dataframes
import pandas as pd

# xarrays
import xarray as xr

# library for turning STAC objects into xarrays
import stackstac

# visualization
from matplotlib import pyplot as plt

# miscellanous
import numpy as np
from IPython.display import display


In [ ]:
# retrieving the relevant STAC Item
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
    )

# bounding box expressed in Lat/Lon
aoi_bounds = (-1.7, 6.6 , -1.5, 6.8)


In [ ]:
item = catalog.get_collection('sentinel-2-l2a').get_item("S2B_MSIL2A_20200516T101559_R065_T30NXN_20200909T222832")

In [ ]:
bands = ['B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B11', 'B12']
FILL_VALUE = 2**16-1

array = stackstac.stack(
                    item,
                    assets = bands,
                    resolution=50,
                    dtype="uint16",
                    fill_value=FILL_VALUE,
                    bounds_latlon=aoi_bounds,
                    )
array

In [ ]:
sub_cube = array.sel(x=slice(643650.,655000.), y=slice(751900,740000) )
sub_cube

In [ ]:
#COMPUTE NDWI INDEX || (NIR - MIR) / (NIR + MIR) || (B08 - B12) / (B08 + B12)

band_mir = sub_cube.sel(band="B12").astype("float")
band_nir = sub_cube.sel(band="B08").astype("float")

NDWI = (band_nir - band_mir) / (band_nir + band_mir)
NDWI.plot.hist()


In [ ]:
#v_min = -0.3
#v_main = 0.6
#NDWI = NDWI.squeeze()
NDWI.plot(vmin= -.3, vmax=.6)

In [ ]:
#COMPUTE EVI INDEX || 2.5 * (NIR - RED) / ((NIR + 6*RED - 7.5*BLUE) + 1) || 2.5 * (B08 - B04) / ((B08 + 6.0 * B04 - 7.5 * B02) + 1.0)

band_blue = sub_cube.sel(band="B02").astype("float")
band_red = sub_cube.sel(band="B04").astype("float")
band_nir = sub_cube.sel(band="B08").astype("float")

EVI = 2.5 * (band_nir - band_red) / ((band_nir + 6.0 * band_red - 7.5 * band_blue) + 1.0)
EVI.plot(robust=True)

In [ ]:
#COMPUTE GNDVI INDEX || (NIR - GREEN) / (NIR + GREEN) || (B08 - B03) / (B08 + B03)

band_green = sub_cube.sel(band="B03").astype("float")
band_nir = sub_cube.sel(band="B08").astype("float")

GNDVI = (band_nir - band_green) / (band_nir + band_green)
GNDVI.plot(robust=True)

In [ ]:
def normalize(data):
  vmin = np.percentile(data,2)
  vmax = np.percentile(data,98)
  #NORMALIZE DATA BETWEEN -1 and 1
  new_data = -1 + ( (data - vmin) / (vmax-vmin) ) * 2
  #REPLACE DATA SMALLER THAN -1 with -1 and REPLACE DATA BIGGER THAN 1 with 1
  new_data = np.clip(new_data, -1, 1.)
  return new_data

NDWI_npy = NDWI.squeeze().to_numpy()
EVI_npy = EVI.squeeze().to_numpy()
GNDVI_npy = GNDVI.squeeze().to_numpy()

NDWI_npy = normalize(NDWI_npy)
EVI_npy = normalize(EVI_npy)
GNDVI_npy = normalize(GNDVI_npy)


rgb = np.stack((NDWI_npy, EVI_npy, GNDVI_npy), axis=-1)
plt.imshow(rgb)